In [27]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier

In [ ]:
class BaseClassifier:
    def __init__(self, data_path):
        # Encapsulation: Data and methods are encapsulated within the class
        self.data = pd.read_csv(data_path)  
        # Split data into train and test
        self.train_data, self.test_data = self.split_data()
        self.train_features, self.train_labels, self.test_features, self.test_labels = self.prepare_features()  # Prepare features

    def split_data(self, test_size=0.3):
        # Abstraction: Hide the complexity of splitting data
        return train_test_split(self.data, test_size=test_size)

    def prepare_features(self):
        # Abstraction: Hide the complexity of feature preparation
        train_features = self.train_data[[
            'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]
        train_labels = self.train_data.Species
        test_features = self.test_data[[
            'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]
        test_labels = self.test_data.Species
        return train_features, train_labels, test_features, test_labels

    def evaluate(self, predictions, true_labels):
        # DRY/KISS: Reusable method for evaluating accuracy
        accuracy = metrics.accuracy_score(predictions, true_labels)
        print(f'The accuracy of the {self.__class__.__name__} is: {accuracy}')
        return accuracy

In [29]:
class SVMClassifier(BaseClassifier):
    def __init__(self, data_path):
        super().__init__(data_path)
        self.model = svm.SVC()  # Composition: Compose an SVM model inside the class

    def train(self):
        self.model.fit(self.train_features, self.train_labels)

    def predict(self):
        return self.model.predict(self.test_features)

In [30]:
class LogisticRegressionClassifier(BaseClassifier):
    def __init__(self, data_path):
        super().__init__(data_path)
        # Composition: Compose a Logistic Regression model inside the class
        self.model = LogisticRegression()

    def train(self):
        self.model.fit(self.train_features, self.train_labels)

    def predict(self):
        return self.model.predict(self.test_features)

In [ ]:
class DecisionTreeClassifierStyle(BaseClassifier):
    def __init__(self, data_path):
        super().__init__(data_path)
        # Composition: Compose a Decision Tree model inside the class
        self.model = DecisionTreeClassifier()

    def train(self):
        self.model.fit(self.train_features, self.train_labels)

    def predict(self):
        return self.model.predict(self.test_features)

In [32]:
class KNNClassifier(BaseClassifier):
    def __init__(self, data_path, n_neighbors=3):
        super().__init__(data_path)
        # Composition: Compose a KNN model inside the class
        self.model = KNeighborsClassifier(n_neighbors=n_neighbors)

    def train(self):
        self.model.fit(self.train_features, self.train_labels)

    def predict(self):
        return self.model.predict(self.test_features)

In [ ]:
def run_classification():
    # Maintainability: The code is modular and easy to maintain
    classifiers = [
        SVMClassifier("./Input_Data/Iris.csv"),
        LogisticRegressionClassifier("./Input_Data/Iris.csv"),
        DecisionTreeClassifierStyle("./Input_Data/Iris.csv"),
        KNNClassifier("./Input_Data/Iris.csv", n_neighbors=3)
    ]

    for classifier in classifiers:
        classifier.train()  # Train the model
        predictions = classifier.predict()  # Make predictions
        # Evaluate accuracy
        classifier.evaluate(predictions, classifier.test_labels)

In [34]:
run_classification()

The accuracy of the SVMClassifier is: 0.9333333333333333
The accuracy of the LogisticRegressionClassifier is: 0.9777777777777777
The accuracy of the DecisionTreeClassifierWrapper is: 0.9555555555555556
The accuracy of the KNNClassifier is: 0.9555555555555556


**Encapsulation**
  - encapsulate the model `train`, `predict`, and `evaluate` logic into a class
  - Ex: The `BaseClassifier` class encapsulates the data loading, splitting, and feature preparation logic. Each subclass (eg. `SVMClassifier`) encapsulates the training, prediction, and evaluation logic for a specific model

**Inheritance**:
  - Each subclass (eg. `SVMClassifier`, `LogisticRegressionClassifier`) inherits from the `BaseClassifier` class and implements the `train` and `predict` methods
  - Ex. The `SVMClassifier` class inherits the `split_data`, `prepare_features`, and `evaluate` methods from `BaseClassifier`

**Favor Composition over Inheritance**:
  - Makes the code more flexible and modular
  - Ex: The `SVMClassifier` class composes an `svm.SVC()` model instead of inheriting from it. Similarly, the `LogisticRegressionClassifier` composes a `LogisticRegression()` model.

#### **Maintainability**:
  - Ex: Adding a new classifier is as simple as creating a new subclass that composes the desired model and implements the `train` and `predict` methods

#### **DRY/KISS**:
  - Code is kept simple and straightforward, adhering to the KISS principle.
  - Ex: Instead of writing evaluation logic in each subclass, the `evaluate` method in `BaseClassifier` is reused by all subclasses.